# Multilayer Neural Network
This is the Exercise for the lesson of Siraj Raval from 2017-01-20

[Link to the lesson - How to Make a Neural Network - Intro to Deep Learning #2](https://www.youtube.com/watch?v=p69khggr1Jo&t=0s) on Youtube

In [ ]:
from numpy import exp, array, random, dot
# The Sigmoid function, which describes an S shaped curve.
# We pass the weighted sum of the inputs through this function to
# normalise them between 0 and 1.
def sigmoid(x):
    return 1 / (1 + exp(-x))

# The derivative of the Sigmoid function.
# This is the gradient of the Sigmoid curve.
# It indicates how confident we are about the existing weight.
def sigmoid_derivative(x):
    return x * (1 - x)


## Layer 
The layer class encouples the layer with the neuron from the network. This is a little step to make the programm more readable.


In [ ]:
class Layer():
    def __init__(self, input_length, output_length):
    
        # We model a single neuron, with input connections of input_length and output connections of output_length.
        # We assign random weights to a [i] x [o] matrix, with values in the range -1 to 1
        # and mean 0.
        self.__weights = 2 * random.random((input_length, output_length)) - 1
        pass

    
    # The neural network thinks.
    def think(self, inputs):
        # Pass inputs through our neural network (our single neuron).
        return sigmoid(dot(inputs, self.__weights))
        
    # adjust the weights
    def adjust_weights(self, adjust):
        self.__weights += adjust
    
    
    def get_weights(self):
        return self.__weights



## The Neural Network
The neural network allowes you in this implementation to set up the size of the input, output and the hidden layer. The algorithm in the init function uses the information to build up the outputs and inputs for the hidden layer. The output size of one layer is the input size for the next and so on. 

The think method is the prediction method from the video on youtube. So I didn't change the name, but include the mechanic to call the hidden layers. 
### The Backpropagation
The backpropagation algorithm need a propagate phase where it get the results for each layer. This is implemented in the propagate method and called in the train method.

After propagate and the calculation of the prediction, the backpropagate phase starts. There are many comments and I hope it is understandable. You have to calculate the error, the delta and adjust them on the layer. 

The post of Andrew Trask [A Neural Network in 13 lines of Python (Part 2 - Gradient Descent)](https://iamtrask.github.io/2015/07/27/python-network-part2/ "Improving our neural network by optimizing Gradient Descent") explaines this very well. 

I removed the loop for iteration on the same data in this method. This must be done by hand. In normaly, you get a bunch of data and iterate 



In [ ]:
class NeuralNetwork():
    def __init__(self, input_length=3, hidden_layers=[8], output_length=1):
        # fill in layers to the neural network
        # Seed the random number generator, so it generates the same numbers
        # every time the program runs.
        random.seed(1)
        # store the layes an an array
        self.__hidden_layers = []
        # the layer algorithm stores a layer into with input and output nodes
        layer_input = input_length
        # Build up hidden layers
        for layer_output in hidden_layers:
            # add new layers
            self.__hidden_layers.append(self.__layer(layer_input, layer_output))
            # the output of the previous layer is the input of the next layer
            layer_input = layer_output
        # define an output layer with the output length
        self.__hidden_layers.append(self.__layer(layer_input, output_length))
        pass
    
    def __layer(self, input_length, output_length):
        return Layer(input_length, output_length)
    
    def last_layer(self):
        return self.__hidden_layers[-1].get_weights()
    
    # prediction function
    def think(self, inputs):
        for layer in self.__hidden_layers:
            inputs = layer.think(inputs)
        return inputs
    
    
    # propagation step
    # move forward in the network -->
    def __propagate(self, inputs):
        outputs = [inputs]
        for layer in self.__hidden_layers:
            inputs = layer.think(inputs)
            outputs.append(inputs)
        return outputs
    
    # backpropagation
    # get the error and the delta of this error
    # adjust the network in the backward direction <--
    def __backpropagation(self, training_set_outputs, propagations):
        # layers for backpropagation
        layers = self.__hidden_layers
        len_layers = len(layers)
        
        # Backpropagation
        deltas = [None]*(len_layers)

        # calculate the error from the output layer and the delta
        error = training_set_outputs - propagations[-1]
        deltas[-1] = error * sigmoid_derivative(propagations[-1])
        # in one loop, we can now calculate the error and the delta for the next iteration
        for position in range(len_layers-1, 0, -1):
            error = dot(deltas[position], layers[position].get_weights().T)
            deltas[position-1] = error * sigmoid_derivative(propagations[position])
        # after calculation of the deltas, we adjust them on the layer
        for position in range(len_layers):
            layers[position].adjust_weights(dot(propagations[position].T, deltas[position]))
        
    # train the network with backpropagation
    def train(self, training_set_inputs, training_set_outputs):
        
        # propagation phase
        propagations = self.__propagate(training_set_inputs)

        # Backpropagation
        self.__backpropagation(training_set_outputs, propagations)
            
        # End for
    # End Method train

In [ ]:
# init the network with the default parameter
neural_network = NeuralNetwork()

# The training set. We have 4 examples, each consisting of 3 input values
# and 1 output value.
training_set_inputs = array([[0, 0, 1], [1, 1, 1], [1, 0, 1], [0, 1, 1]])
training_set_outputs = array([[0, 1, 1, 0]]).T


print("New synaptic weights after training: ")
print(neural_network.last_layer())

# Train the neural network using a training set.
# Do it 10,000 times and make small adjustments each time.
for i in range(0, 1000):
    neural_network.train(training_set_inputs, training_set_outputs)

    
print("New synaptic weights after training: ")
print(neural_network.last_layer())

# Test the neural network with a new situation.
print("Considering new situation [1, 0, 0] -> ?: ")
print(neural_network.think(array([1, 0, 0])))